In [3]:
import importlib
import os
import zipfile as zf
import duckdb
import pandas as pd
import numpy as np
import pyarrow as pa
import cvm

importlib.reload(cvm)
pd.set_option("max_colwidth", 15)
pd.set_option("max_rows", 4)

In [10]:
files_names = sorted(os.listdir('./data/raw/'))
files_names

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip',
 'dfp_cia_aberta_2021.zip',
 'itr_cia_aberta_2020.zip',
 'itr_cia_aberta_2021.zip',
 'itr_cia_aberta_2022.zip']

In [11]:
def df_format(df) -> pd.DataFrame:
    df.VERSAO = df.VERSAO.astype(np.int8)  # unique -> ['3', '2', '4', '1', '7', '5', '6', '9', '8']
    df.CD_CVM = df.CD_CVM.astype(np.int32)  # max < 600_000
    df.VL_CONTA = df.VL_CONTA.astype(float)

    # df.MOEDA.value_counts()
    # REAL    43391302
    df.drop(columns=['MOEDA'], inplace=True)
    
    # df.ESCALA_MOEDA.value_counts()
    # MIL        40483230
    # UNIDADE     2908072
    df.ESCALA_MOEDA = df.ESCALA_MOEDA.map({'MIL': 1000, 'UNIDADE': 1})

    # unit base currency
    df.VL_CONTA = df.VL_CONTA * df.ESCALA_MOEDA
    df.drop(columns=['ESCALA_MOEDA'], inplace=True)

    # df.ST_CONTA_FIXA.unique() -> ['S', 'N']
    df.ST_CONTA_FIXA = df.ST_CONTA_FIXA.map({'S': True, 'N': False})

    # df.ORDEM_EXERC.unique() -> ['PENÚLTIMO', 'ÚLTIMO']
    df.ORDEM_EXERC = df.ORDEM_EXERC.map({'ÚLTIMO': 0, 'PENÚLTIMO': -1})
    df.ORDEM_EXERC = df.ORDEM_EXERC.astype(np.int8)

    # BPA, BPP and DFC files have no DT_INI_EXERC column
    if 'DT_INI_EXERC' in df.columns:
        df.DT_INI_EXERC = pd.to_datetime(df.DT_INI_EXERC)
    else:
        # column_order.remove('DT_INI_EXERC')
        df['DT_INI_EXERC'] = pd.NaT
    if 'COLUNA_DF' not in df.columns: df['COLUNA_DF'] = np.nan

    column_order = [
        'CD_CVM', 'CNPJ_CIA', 'DENOM_CIA', 'GRUPO_DFP', 'VERSAO', 'DT_REFER',
        'DT_INI_EXERC', 'DT_FIM_EXERC', 'ORDEM_EXERC', 'CD_CONTA', 'DS_CONTA',
        'ST_CONTA_FIXA', 'COLUNA_DF', 'VL_CONTA']

    df = df[column_order]
    return df 

In [12]:
date_columns = ['DT_REFER', 'DT_FIM_EXERC']
kwargs = {
    'sep': ';',
    'encoding': 'iso-8859-1',
    'dtype': str,
    'parse_dates': date_columns}

In [19]:
# full scan = 8m 10s -> 5.6 GB df memory space
# formated colums = 4m 30s -> 3.5 GB df memory space
# zstd -> 16.5 seg
# lz4 -> 15.4 seg
# df_all = pd.DataFrame()
for parent_file_name in files_names[:]:
    print(parent_file_name)
    file_path = f'./data/raw/{parent_file_name}'
    parent_file = zf.ZipFile(file_path)
    # print(parent_file.namelist())
    df_year = pd.DataFrame()
    child_file_names = parent_file.namelist()
    for child_file_name in child_file_names[1:]:
        # print(child_parent_file_name)
        child_file = parent_file.open(child_file_name)
        df_child = pd.read_csv(child_file, **kwargs)
        df_child = df_format(df_child)        
        # df_all = pd.concat([df_all, df_child], ignore_index=True)
        df_year = pd.concat([df_year, df_child], ignore_index=True)
    sort_by = [
        'CD_CVM', 'GRUPO_DFP', 'VERSAO', 'ORDEM_EXERC', 'DT_REFER', 'CD_CONTA']
    df_year.sort_values(by=sort_by, ignore_index=True, inplace=True)
    df_year.to_parquet(
        f'./data/processed/{parent_file_name[:-4]}.parquet',
        index=False,
        compression='zstd')
    # df_year.to_feather(f'./data/processed/{parent_file_name[:-4]}.feather')

dfp_cia_aberta_2010.zip
dfp_cia_aberta_2011.zip
dfp_cia_aberta_2012.zip
dfp_cia_aberta_2013.zip
dfp_cia_aberta_2014.zip
dfp_cia_aberta_2015.zip
dfp_cia_aberta_2016.zip
dfp_cia_aberta_2017.zip
dfp_cia_aberta_2018.zip
dfp_cia_aberta_2019.zip
dfp_cia_aberta_2020.zip
dfp_cia_aberta_2021.zip
itr_cia_aberta_2020.zip
itr_cia_aberta_2021.zip
itr_cia_aberta_2022.zip


In [6]:
files_names = sorted(os.listdir('./data/processed/'))
files_names

['dfp_cia_aberta_2010.parquet',
 'dfp_cia_aberta_2011.parquet',
 'dfp_cia_aberta_2012.parquet',
 'dfp_cia_aberta_2013.parquet',
 'dfp_cia_aberta_2014.parquet',
 'dfp_cia_aberta_2015.parquet',
 'dfp_cia_aberta_2016.parquet',
 'dfp_cia_aberta_2017.parquet',
 'dfp_cia_aberta_2018.parquet',
 'dfp_cia_aberta_2019.parquet',
 'dfp_cia_aberta_2020.parquet',
 'dfp_cia_aberta_2021.parquet',
 'itr_cia_aberta_2020.parquet',
 'itr_cia_aberta_2021.parquet',
 'itr_cia_aberta_2022.parquet']

In [9]:
df = pd.DataFrame()
for file_name in files_names:
    dfy = pd.read_parquet(f'./data/processed/{file_name}', filters=[('CD_CVM','==',9512)])
    df = pd.concat([df, dfy], ignore_index=True)
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,None,3.504189e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,NaT,2009-12-31,-1,1.01,Ativo Circu...,True,None,7.437357e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-01-01,2021-09-30,0,3.99.02.02,PN,False,None,5.760000e+03
37113,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-07-01,2021-09-30,0,3.99.02.02,PN,False,None,2.390000e+03


In [34]:
pd.read_parquet('./data/processed/dfp_cia_aberta_2020.parquet')['CD_CVM'].nunique()

719

In [20]:
df = pd.read_parquet('./data/processed/dfp_cia_aberta_2020.parquet', filters=[('CD_CVM','==',9512)])
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,NaT,2019-12-31,-1,1,Ativo Total,True,None,9.260110e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,NaT,2019-12-31,-1,1.01,Ativo Circu...,True,None,1.121010e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,9512,33.000.167/...,PETROLEO BR...,DF Individu...,2,2020-12-31,2020-01-01,2020-12-31,0,3.99.02.01,ON,False,None,5.400000e+02
2187,9512,33.000.167/...,PETROLEO BR...,DF Individu...,2,2020-12-31,2020-01-01,2020-12-31,0,3.99.02.02,PN,False,None,5.400000e+02


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2188 entries, 0 to 2187
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         2188 non-null   int32         
 1   CNPJ_CIA       2188 non-null   object        
 2   DENOM_CIA      2188 non-null   object        
 3   GRUPO_DFP      2188 non-null   object        
 4   VERSAO         2188 non-null   int8          
 5   DT_REFER       2188 non-null   datetime64[ns]
 6   DT_INI_EXERC   1488 non-null   datetime64[ns]
 7   DT_FIM_EXERC   2188 non-null   datetime64[ns]
 8   ORDEM_EXERC    2188 non-null   int8          
 9   CD_CONTA       2188 non-null   object        
 10  DS_CONTA       2188 non-null   object        
 11  ST_CONTA_FIXA  2188 non-null   bool          
 12  COLUNA_DF      832 non-null    object        
 13  VL_CONTA       2188 non-null   float64       
dtypes: bool(1), datetime64[ns](3), float64(1), int32(1), int8(2), object(6)


In [22]:
import pyarrow.parquet as pq
table = pq.read_table(
    "./data/processed/",
    use_legacy_dataset=False,
    filters=[('CD_CVM','==',9512)])
df = table.to_pandas()
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,None,3.504189e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,NaT,2009-12-31,-1,1.01,Ativo Circu...,True,None,7.437357e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-01-01,2021-09-30,0,3.99.02.02,PN,False,None,5.760000e+03
37113,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-07-01,2021-09-30,0,3.99.02.02,PN,False,None,2.390000e+03


In [29]:
metadata = pq.read_metadata('./data/processed/dfp_cia_aberta_2020.parquet')
metadata

  created_by: parquet-cpp-arrow version 6.0.1
  num_columns: 14
  num_rows: 1179386
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 7350

In [33]:
metadata.row_group(0).column(5)

  file_offset: 36506
  file_path: 
  physical_type: INT64
  num_values: 1179386
  path_in_schema: DT_REFER
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 2020-02-29 00:00:00
      max: 2020-12-31 00:00:00
      null_count: 0
      distinct_count: 0
      num_values: 1179386
      physical_type: INT64
      logical_type: Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false)
      converted_type (legacy): NONE
  compression: ZSTD
  encodings: ('PLAIN_DICTIONARY', 'PLAIN', 'RLE')
  has_dictionary_page: True
  dictionary_page_offset: 36208
  data_page_offset: 36279
  total_compressed_size: 298
  total_uncompressed_size: 280

In [23]:
import pyarrow.dataset as ds
dataset = ds.dataset('./data/processed/', format='parquet')
dataset.files

['./data/processed/dfp_cia_aberta_2010.parquet',
 './data/processed/dfp_cia_aberta_2011.parquet',
 './data/processed/dfp_cia_aberta_2012.parquet',
 './data/processed/dfp_cia_aberta_2013.parquet',
 './data/processed/dfp_cia_aberta_2014.parquet',
 './data/processed/dfp_cia_aberta_2015.parquet',
 './data/processed/dfp_cia_aberta_2016.parquet',
 './data/processed/dfp_cia_aberta_2017.parquet',
 './data/processed/dfp_cia_aberta_2018.parquet',
 './data/processed/dfp_cia_aberta_2019.parquet',
 './data/processed/dfp_cia_aberta_2020.parquet',
 './data/processed/dfp_cia_aberta_2021.parquet',
 './data/processed/itr_cia_aberta_2020.parquet',
 './data/processed/itr_cia_aberta_2021.parquet',
 './data/processed/itr_cia_aberta_2022.parquet']

In [12]:
dataset.to_table().to_pandas()

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,94,92.693.019/...,PANATLANTIC...,DF Consolid...,2,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,None,211293000.0
1,94,92.693.019/...,PANATLANTIC...,DF Consolid...,2,2010-12-31,NaT,2009-12-31,-1,1.01,Ativo Circu...,True,None,164307000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17608709,25810,31.701.408/...,ATHENA SAÚD...,DF Individu...,1,2022-03-31,2022-01-01,2022-03-31,0,3.99.02,Lucro Diluí...,False,None,0.0
17608710,25810,31.701.408/...,ATHENA SAÚD...,DF Individu...,1,2022-03-31,2022-01-01,2022-03-31,0,3.99.02.01,ON,False,None,-62.6


In [24]:
dataset.to_table(filter=ds.field('CD_CVM') == 9512).to_pandas()

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,None,3.504189e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,NaT,2009-12-31,-1,1.01,Ativo Circu...,True,None,7.437357e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-01-01,2021-09-30,0,3.99.02.02,PN,False,None,5.760000e+03
37113,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-07-01,2021-09-30,0,3.99.02.02,PN,False,None,2.390000e+03


In [27]:
sql = """
SELECT *
FROM parquet_scan('data/processed/*.parquet')
WHERE CD_CVM = 9512;
"""
df = duckdb.query(sql).to_df()
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,NaT,2019-12-31,-1,1,Ativo Total,True,NaN,9.260110e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,2,2020-12-31,NaT,2019-12-31,-1,1.01,Ativo Circu...,True,NaN,1.121010e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.02.01,ON,False,NaN,1.790000e+03
37113,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.02.02,PN,False,NaN,1.790000e+03


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37114 entries, 0 to 37113
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         37114 non-null  int32         
 1   CNPJ_CIA       37114 non-null  object        
 2   DENOM_CIA      37114 non-null  object        
 3   GRUPO_DFP      37114 non-null  object        
 4   VERSAO         37114 non-null  int32         
 5   DT_REFER       37114 non-null  datetime64[ns]
 6   DT_INI_EXERC   25026 non-null  datetime64[ns]
 7   DT_FIM_EXERC   37114 non-null  datetime64[ns]
 8   ORDEM_EXERC    37114 non-null  int32         
 9   CD_CONTA       37114 non-null  object        
 10  DS_CONTA       37114 non-null  object        
 11  ST_CONTA_FIXA  37114 non-null  bool          
 12  COLUNA_DF      13916 non-null  object        
 13  VL_CONTA       37114 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64(1), int32(3), object(6)
memory 

In [10]:
df[df.COLUNA_DF.notnull()]

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
372,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,2009-01-01,2009-12-31,-1,5.01,Saldos Inic...,True,Capital Soc...,7.896669e+10
373,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31,2009-01-01,2009-12-31,-1,5.01,Saldos Inic...,True,Reservas de...,5.148570e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35928,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-01-01,2021-09-30,0,5.07,Saldos Finais,True,Outros Resu...,-2.110000e+08
35929,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30,2021-01-01,2021-09-30,0,5.07,Saldos Finais,True,Patrimônio ...,3.690770e+11


In [2]:
con = duckdb.connect(database='data/db/my-db.duckdb', read_only=False)
con

In [3]:
sql = "CREATE TABLE teste_tb AS SELECT * FROM parquet_scan('data/processed/*.parquet');"
con.execute(sql).fetchall()

[(17608711,)]

In [31]:
df.GRUPO_DFP.unique()
# df

array(['DF Consolidado - Balanço Patrimonial Ativo',
       'DF Individual - Balanço Patrimonial Ativo',
       'DF Consolidado - Balanço Patrimonial Passivo',
       'DF Individual - Balanço Patrimonial Passivo',
       'DF Consolidado - Demonstração do Fluxo de Caixa (Método Direto)',
       'DF Individual - Demonstração do Fluxo de Caixa (Método Direto)',
       'DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Individual - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Consolidado - Demonstração das Mutações do Patrimônio Líquido',
       'DF Individual - Demonstração das Mutações do Patrimônio Líquido',
       'DF Consolidado - Demonstração de Resultado Abrangente',
       'DF Individual - Demonstração de Resultado Abrangente',
       'DF Consolidado - Demonstração do Resultado',
       'DF Individual - Demonstração do Resultado',
       'DF Consolidado - Demonstração de Valor Adicionado',
       'DF Individual - Demonstração de Valor A

In [11]:
df = cvm.load_metadata()
df

,DT_REFER,VERSAO,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB
0,2010-12-31,1,1023,DFP,4822,2011-02-17
1,2010-12-31,2,1023,DFP,4823,2011-02-17
...,...,...,...,...,...,...
14179,2021-09-30,1,21091,ITR,109262,2021-10-27
14180,2022-03-31,1,25810,ITR,110634,2021-12-08
